<a href="https://colab.research.google.com/github/rayaneghilene/BERT_classification/blob/main/DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of the performance DistilBERT on a classification task

Modify the following variables based on your data
* **DATAPATH**: is the path to your dataset in your environment

In [ ]:
DATAPATH = '/content/Multitarget-CONAN (2).csv'

## Data  

In [3]:
# Install and import the required libraries
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off


In [ ]:
!pip install nltk --progress-bar off
!pip install keras --progress-bar off
!pip install tensorflow --progress-bar off
!pip install tensorflow_hub --progress-bar off
!pip install transformers --progress-bar off

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss

## I - DistilBERT

In [7]:
import tensorflow as tf
import pandas as pd
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split

# Data preprocessing
df = pd.read_csv(DATAPATH)
df["label"] = df["TARGET"].astype("category").cat.codes  # label encoding
data_texts = df["HATE_SPEECH"].to_list()
data_labels = df["label"].to_list()
train_texts, test_texts, train_labels, test_labels = train_test_split(data_texts, data_labels, test_size=0.2)

# Tokenisation
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

# DistilBERT Model
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8)  # Replace 8 with your number of classes

# Optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


train_dataset = train_dataset.batch(16)
test_dataset = test_dataset.batch(16)

# Training
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Save the model
model.save_pretrained("./distilbert-fine-tuned")

# Load the fine-tuned model
model = TFDistilBertForSequenceClassification.from_pretrained("./distilbert-fine-tuned")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
251/251 [==============================] - 90s 225ms/step - loss: 0.4271 - accuracy: 0.8986 - val_loss: 0.1432 - val_accuracy: 0.9670
Epoch 2/3
251/251 [==============================] - 48s 190ms/step - loss: 0.1104 - accuracy: 0.9690 - val_loss: 0.1450 - val_accuracy: 0.9630
Epoch 3/3
251/251 [==============================] - 47s 186ms/step - loss: 0.0803 - accuracy: 0.9758 - val_loss: 0.1231 - val_accuracy: 0.9710


Some layers from the model checkpoint at ./distilbert-fine-tuned were not used when initializing TFDistilBertForSequenceClassification: ['dropout_79']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./distilbert-fine-tuned and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Testing on a custom Prompt

In [14]:
label_encoding = df["TARGET"].astype("category").cat.codes
label_mapping = dict(enumerate(df["TARGET"].astype("category").cat.categories))


custom_prompt = "this message presents hate against muslims"
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
prompt_encoding = tokenizer(custom_prompt, truncation=True, padding=True, return_tensors="tf")

predictions = model(prompt_encoding["input_ids"]).logits
probabilities = tf.nn.softmax(predictions, axis=-1).numpy()
predicted_class = tf.argmax(predictions, axis=-1).numpy()[0]
predicted_label = label_mapping[predicted_class]


print("Class Probabilities:", probabilities)
print("Predicted Label:", predicted_label)

Class Probabilities: [[1.5377958e-04 2.5136684e-04 1.6974899e-04 6.6160556e-04 9.8733109e-01
  2.6529352e-04 1.5257922e-04 1.1014453e-02]]
Predicted Label: MUSLIMS
